In [2]:
import os
os.chdir('/kaggle/input/myfunctions')
from functions import *
from myutils import *
import pandas as pd
import numpy as np
import pickle
from functions import *
os.chdir('/kaggle/input/mydatasets2')
from sklearn.model_selection import train_test_split
import xgboost as xgb
from tqdm import tqdm

In [ ]:
time_steps = 20 # Time steps to consider
df_path = '/kaggle/input/mydatasets2/training_set1.pkl'

In [4]:
data, labels = preprocess_data_via_close_values(df_path, time_steps=time_steps, shuffle=True, split=False)
print(data.shape)
print(labels.shape)

Columns of each dataframe Index(['Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')
Shape of each array in the list (2181, 20)
Label Classes ['decrease' 'increase' 'no big change']
Data available for each label - increase, decrease and no big change 2107545 2132975 121480
Combined array shape -  (4362000, 20)
Combined labels shape -  (4362000,)
Shuffled data shape -  (4362000, 20)
(4362000, 20)
(4362000,)


In [6]:
test_size=0.2
random_state=42
x_train, x_valid, y_train, y_valid = train_test_split(data, labels, test_size=test_size, random_state=random_state)
num_bagging_splits = 100
models_dict = {}
params = {'objective':'multi:softmax', 'eval_metric':'merror', 'num_class':'3', 'eta':'0.1', 'max_depth':'5'}
num_boost_rounds = 100
scaling = 2.5
for i in tqdm(range(num_bagging_splits)):
    indices = np.arange(0, len(x_train))
    np.random.shuffle(indices)
    slice_val = int(len(indices)//scaling)
    indices_selected = indices[:slice_val]
    x_tr = x_train[indices_selected]
    y_tr = y_train[indices_selected]
    xg_train = xgb.DMatrix(x_tr, label=y_tr)
    models_dict[i] = xgb.train(params, xg_train, num_boost_round=num_boost_rounds)

100%|██████████| 100/100 [59:37<00:00, 35.77s/it]


In [7]:
def xg_multi_predict(x, y, models_dict = models_dict):
    xg_valid = xgb.DMatrix(x, label=y)
    y_pred = np.zeros_like(y, dtype=np.float32)
    # y_pred = models_dict[0].predict(xg_valid)
    for xg_model in models_dict.values():
        y_pred += xg_model.predict(xg_valid)
    y_pred = y_pred/len(models_dict)
    return y_pred

In [8]:
metric_keys = ("accuracy", "precision", "specificity", "recall", "f1score")
y_train_pred = xg_multi_predict(x = x_train, y = y_train, models_dict = models_dict)
y_train_pred = np.round(y_train_pred)

In [9]:
train_metrics = dict(zip(metric_keys, metric_calculations(y_train_pred, y_train, set_="Train set")))

For Train set
  Accuracy: 41.29324277854195,
  Precision: 50.27566235750355,
  Recall: 42.29500183880057,
  Specificity: 39.85285970646096,
  F1score: 45.94131953829983


In [10]:
y_pred = xg_multi_predict(x = x_valid, y = y_valid, models_dict = models_dict)
y_pred = np.round(y_pred)

In [11]:
valid_metrics = dict(zip(metric_keys, metric_calculations(y_pred, y_valid, set_="Validation set")))

For Validation set
  Accuracy: 40.88090325538744,
  Precision: 49.83482491325802,
  Recall: 41.94274048428634,
  Specificity: 39.35572156561179,
  F1score: 45.54945687135334


In [12]:
df_path_test = "/kaggle/input/mydatasets2/training_set11.pkl"
x_test, y_test = preprocess_data_via_close_values(df_path_test, time_steps=20, shuffle=True, split=False)

Columns of each dataframe Index(['Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')
Shape of each array in the list (868, 20)
Label Classes ['decrease' 'increase' 'no big change']
Data available for each label - increase, decrease and no big change 1036659 1034361 52976
Combined array shape -  (2123996, 20)
Combined labels shape -  (2123996,)
Shuffled data shape -  (2123996, 20)


In [13]:
y_pred_test = xg_multi_predict(x = x_test, y = y_test, models_dict = models_dict)
y_pred_test = np.round(y_pred_test)

In [14]:
test_metrics = dict(zip(metric_keys, metric_calculations(y_pred_test, y_test, set_="Test set")))

For Test set
  Accuracy: 36.79361919702297,
  Precision: 49.81335755066485,
  Recall: 39.74922936898867,
  Specificity: 31.75706126898189,
  F1score: 44.215841525370735


In [15]:
output_keys = ["models", "train_metrics", "validation_metrics", "test metrics"]
output_values = [models_dict, train_metrics, valid_metrics, test_metrics]
output_dict = dict(zip(output_keys, output_values))

In [17]:
output_file_path = f"/kaggle/working/models_xg_bagging_{num_bagging_splits}_{time_steps}.pkl"
write_file_to_pickle(output_file_path, output_dict)